In [1]:
from lines_generator import dataGenerator
import numpy as np
import keras
from keras import Sequential
from keras.layers import Dense,Conv2D, MaxPooling2D, Flatten

Using TensorFlow backend.


In [2]:
patche_size = 17
input_dim = patche_size**2

## ANN Generator

In [3]:
gen_train = dataGenerator(50,'train',prob=0.5)
gen_test = dataGenerator(10,'test',prob=0.5)

In [7]:
# create model
# activations = ['relu','softmax','sigmoid']
model = Sequential()
model.add(Dense(1000, input_dim=input_dim, activation='sigmoid'))
model.add(Dense(1000, activation='sigmoid'))
model.add(Dense(2, activation='softmax'))

In [8]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [9]:
model.fit_generator(gen_train, samples_per_epoch=64, nb_epoch=20)

/home/cordeiro/env/py3im/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  """Entry point for launching an IPython kernel.
/home/cordeiro/env/py3im/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., steps_per_epoch=64, epochs=20)`
  """Entry point for launching an IPython kernel.


Epoch 1/20
64/64 [==============================] - 2s 27ms/step - loss: 0.3941 - acc: 0.9248
Epoch 2/20
64/64 [==============================] - 2s 24ms/step - loss: 0.1014 - acc: 0.9727
Epoch 3/20
64/64 [==============================] - 2s 24ms/step - loss: 0.0520 - acc: 0.9795
Epoch 4/20
64/64 [==============================] - 2s 24ms/step - loss: 0.0974 - acc: 0.9844
Epoch 5/20
64/64 [==============================] - 2s 24ms/step - loss: 0.0799 - acc: 0.9844
Epoch 6/20
64/64 [==============================] - 2s 26ms/step - loss: 0.1465 - acc: 0.9717
Epoch 7/20
64/64 [==============================] - 2s 26ms/step - loss: 0.0660 - acc: 0.9766
Epoch 8/20
64/64 [==============================] - 2s 27ms/step - loss: 0.0446 - acc: 0.9844
Epoch 9/20
64/64 [==============================] - 2s 28ms/step - loss: 0.0771 - acc: 0.9805
Epoch 10/20
64/64 [==============================] - 1s 23ms/step - loss: 0.0368 - acc: 0.9893
Epoch 11/20
64/64 [==============================] - 2s 27m

In [10]:
score = model.evaluate_generator(gen_test,steps=128)
score

[0.04295787410410412, 0.99267578125]

In [11]:
pred_test = model.predict_generator(gen_test,steps=128,verbose=1)

128/128 [==============================] - 0s 3ms/step


In [12]:
pred_test

array([[1.7437924e-02, 9.8256207e-01],
       [1.0000000e+00, 9.4904253e-09],
       [1.0000000e+00, 2.6452112e-09],
       ...,
       [1.0000000e+00, 1.7994246e-08],
       [1.0000000e+00, 3.6045578e-09],
       [1.9127239e-02, 9.8087275e-01]], dtype=float32)

## Save

In [13]:
model.save('models/model2.h5')